In [76]:
######## PLOTTING USING 39000+ TILES DICTIONAIRY #############################################

import rasterio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.mask import mask
from shapely.geometry import box
from shapely.geometry import Polygon
import geopandas as gpd
from fiona.crs import from_epsg
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import geopandas as gpd
import requests
import json
import os
from rasterio.transform import from_bounds
import gdal
import shapely
import re
import rasterio
from rasterio.plot import show
from rasterio import plot
# import richdem as rd
# import geopy
from shapely.geometry import point
from PIL import Image
import rasterio as rio
from rasterio.plot import show
#import pycrs

In [77]:
# find coordinates of address you are searching

#address = "Sint-Pietersvliet 7, 2000 Antwerpen"  
#address = "Francis Wellesplein 1, 2018 Antwerpen" 
#address = "Eigenaarsstraat 13, 2350 Vosselaar"  
#address = "De Keyserlei 50, 2000 Antwerpen"
#address = "Narcislaan 13, 2900 Schoten" 
#address = "Werverbos 78, 2930 Brasschaat"
#address = "Pareloesterlaan 15, 8400 Oostende"
address = "De Tomermaat 25, 2930 Brasschaat" 
#address = "Huidevetterslaan 10, 3500 Hasselt" 
#address = "Serafijnstraat 8, 9000 Gent" 
#address = "Veldstraat 45, 9630 Zwalm" 

def get_coordinates(address: str) -> (int, int):
    req = requests.get(f"http://loc.geopunt.be/geolocation/location?q={address}&c=1")
    return (req.json()["LocationResult"][0]["Location"]["X_Lambert72"],
            req.json()["LocationResult"][0]["Location"]["Y_Lambert72"])


coordinaten = get_coordinates(address)
x = coordinaten[0]
y = coordinaten[1]
print("x coordinaat = ", coordinaten[0])  # X
print("y coordinaat = ", coordinaten[1])  # Y

x coordinaat =  159894.08
y coordinaat =  220174.04


In [78]:
#read the boundvalues from the CSV into pandas framework
path="D:\\PROJECT_BECODE\\bounds_full.csv"

boundaries = pd.read_csv(path, index_col=0)

boundaries.columns = ["Left", "Bottom", "Right", "Top"] #make columns names

boundaries["Left"] = boundaries["Left"].apply(lambda x : x[1:]) #remove '('
boundaries["Left"] = pd.to_numeric(boundaries["Left"]) #filling in the columns #to numeric becuase you can't calculate with strings
boundaries["Bottom"] = pd.to_numeric(boundaries["Bottom"]) #filling in the columns
boundaries["Right"] = pd.to_numeric(boundaries["Right"]) #filling in the columns
boundaries["Top"] = boundaries["Top"].apply(lambda x : x[:-1]) #remove ')'
boundaries["Top"] = pd.to_numeric(boundaries["Top"]) #filling in the columns

#boundaries.head()


In [79]:
#compare coordinate values to values of pandaframe

path = ""

for i in range(1, 39294): #loop over all the bound-values of te tiff files out of the csv in the pandas framework
        left = boundaries.iloc[i]["Left"]
        right = boundaries.iloc[i]["Right"]
        bottom = boundaries.iloc[i]["Bottom"]
        top = boundaries.iloc[i]["Top"]
        #print("tile ",i, " --->", left , " ", right, " ", bottom, " ", top)

        if x > left and x < right and y > bottom and y < top: #check if coordinates are in the bounds
            print("tile number ->" , i)
            print("left " , left)
            print("right ", right)
            print("bottom " , bottom)
            print("top ", top)

            
            path = "D:\\PROJECT_BECODE\\DSM_split\\tile_{}.tif".format(i+2) #is always 2 tiff files to short for some reason
            #rasterio.plot.show(file) 
            break
            
            
            #################### PROJECT HOUSE #################################################""

            data = rio.open(path)
            address_regx = re.compile("^([A-z- ]+)\s(\d+),\s(\d+)\s([A-z]+)")
            result = address_regx.search(address)
            street = result.group(1)
            print(street)
            nb = result.group(2)
            print(nb)
            pc = result.group(3)
            print(pc)
            city = result.group(4)
            print(city)

            req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
            objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]

            req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
            objectId = req["gebouw"]["objectId"]

            req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()
            polygon = [req["geometriePolygoon"]["polygon"]] 
            print("polygon -> ", polygon)
        
            crop_img, crop_transform = mask(dataset=data, shapes=polygon, crop=True, indexes=1, nodata = 0, filled=True)
            print("crop image -> " , crop_img)

            fig = go.Figure(data=go.Surface(z=crop_img, colorscale='YlOrRd'))
            fig.show()

            #break


            

tile number -> 4175
left  159000.0
right  160000.0
bottom  220000.0
top  220500.0


In [80]:
#################### PROJECT HOUSE ####################w

data = rio.open(path)
address_regx = re.compile("^([A-z- ]+)\s(\d+),\s(\d+)\s([A-z]+)")
result = address_regx.search(address)
street = result.group(1)
print(street)
nb = result.group(2)
print(nb)
pc = result.group(3)
print(pc)
city = result.group(4)
print(city)

req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/adresmatch?gemeentenaam={city}&straatnaam={street}&huisnummer={nb}&postcode={pc}").json()
objectId = req["adresMatches"][0]["adresseerbareObjecten"][0]["objectId"]

req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouweenheden/{objectId}").json()
objectId = req["gebouw"]["objectId"]

req = requests.get(f"https://api.basisregisters.dev-vlaanderen.be/v1/gebouwen/{objectId}").json()
polygon = [req["geometriePolygoon"]["polygon"]] 
print("polygon -> ", polygon)
        
crop_img, crop_transform = mask(dataset=data, shapes=polygon, crop=True, indexes=1, nodata = 0, filled=True)
print("crop image -> " , crop_img)

fig = go.Figure(data=go.Surface(z=crop_img, colorscale='YlOrRd'))
fig.show()






De Tomermaat
25
2930
Brasschaat
polygon ->  [{'coordinates': [[[159902.5781300962, 220172.94562679157], [159901.24846609682, 220178.7261067927], [159887.56117008626, 220175.60956279188], [159886.16238608956, 220175.2910987921], [159886.31854608655, 220174.64546679333], [159886.10760208964, 220173.9399307929], [159885.90894608945, 220172.7244427912], [159885.8592820838, 220171.50281079113], [159886.06817808747, 220170.28629878908], [159887.1582260877, 220170.54639478773], [159887.40501008928, 220169.49116278812], [159902.5781300962, 220172.94562679157]]], 'type': 'Polygon'}]
crop image ->  [[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
   0.    0.    0.   20.13  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   20.93
  20.9  21.55 21.46 20.85 16.43  0.  ]
 [ 0.    0.    0.    0.    0.    0.   21.16 20.92 20.93 20.94 21.27 21.36
  21.25 21.16 21.05 20.91 20.65  0.  ]
 [ 0.    0.   16.95 20.75 21.28 21.09 21.4  21.32 21.23 21.09 20.91